In [2]:
import pandas as pd
tables= pd.read_csv("tables_RIP.csv", sep=",",encoding='utf-8',na_values="")
tables.rename(columns={'Table correspondante':'tab','Séjour / ACE':'struct', 'Public / privé': 'champ','Observation CNAMTS': 'observation' }, inplace=True)

tablestruct={
    'tables':tables['tab'].to_list(),
    'structure':tables['struct'].to_list(),
    'champ': tables['champ'].to_list(), 
    'observation': tables['observation'].to_list()
}

In [7]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("RIP.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'T_RIPaaTP': 'Table','Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted','Séjour / ACE':'struct','Public / privé': 'champ',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "Char": "string",
    "char":"string",
    "Num":"integer",
    "num":"integer",
    "date":"date",
    "Date":"date",
    "":"string" # cas où le type est vide
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"

def strdate(x):
  if 'MOI' in x:
    return "date"
  return "string"

df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i]=='number':
    df['type'][i]=numbint(df['length'][i])  
  elif df['type'][i]=='string':
    df['type'][i]=strdate(df['variables'][i])
    

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI RIP'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
      tableexist['champ'].append(df['champ'][i])
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']


  for tables in tablestruct['tables']:
    #séjours Public et Privé avec les RPSA(RSA)
    if tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Public et Privé" : 
      if table=='T_RIPaaRSA':
        schema.descriptor['primaryKey']= [
        "ETA_NUM_EPMSI",
        "RIP_NUM",
        "SEQ_SEQ_NUM"
        ]
        schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM_EPMSI"],
                                           "reference": {
                                             "resource": "T_RIPaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
  
      elif table=="T_RIPaaE": # Table des établissements
        schema.descriptor['primaryKey']= [
            "ETA_NUM"
            ]
 
     # On identifie un séjour par 2 clés de jointure et une séquence par 3 clés de jointure                                    
     #1 Tables dont la jointure avec la table principale RSA se fait par le Séjour : 
      elif table=='T_RIPaaC':
        schema.descriptor['primaryKey']= [
        "ETA_NUM_EPMSI",
        "RIP_NUM"
        ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RIP_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_RIPaaRSA",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RIP_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                         ,
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }]
        
      elif table in ['T_RIPaaCCAM', 'T_RIPaaS']:
        schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RIP_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_RIPaaFB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RIP_NUM"
                                            ]
                                            }
                                       }
                                      ]
     #2 Tables dont la jointure avec la table principale RPSA se fait par la séquence :

      elif table in ['T_RIPaaRSAD','T_RIPaaTP']:
        schema.descriptor['foreignKeys']=[{"fields":[
                                            "ETA_NUM_EPMSI",
                                            "RIP_NUM",
                                            "SEQ_SEQ_NUM"
                                        ],
                                        "reference": {
                                              "resource": "T_RIPaaRSA",
                                              "fields": [
                                                  "ETA_NUM_EPMSI",
                                                  "RIP_NUM",
                                                  "SEQ_SEQ_NUM"
                                              ]
                                              }
                                        }
                                        ]      
      
        
      
      else: # pas forcément nécéssaire
        schema.descriptor['foreignKeys']=[
            {
                "fields": [
                    "ETA_NUM_EPMSI",
                    "RIP_NUM"

                ],
                "reference": {
                    "resource": "T_RIPaaRSA",
                    "fields": [
                        "ETA_NUM_EPMSI",
                        "RIP_NUM"
                    ]
                }
            }
        ]

   #Tables COVID et/ou Public
    elif table in ["T_RIPaaSUP_VAC", "T_RIPaaSUP_DSC", ]: 
      schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_RIPaaE",
                                                "fields": [
                                                    "ETA_NUM"
                                                ]
                                                }
                                          }
                                          ]
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Privé" : #séjours Privés: tables de facturation en établissement Privé
      if table=='T_RIPaaFA':
        schema.descriptor['primaryKey']= [
            "ETA_NUM_EPMSI",
            "RIP_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM_EPMSI",
                                              "RIP_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_RIPaaC",
                                                "fields": [
                                                    "ETA_NUM_EPMSI",
                                                    "RIP_NUM"
                                                ]
                                                }
          }]
                                          
      else:
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM_EPMSI",
                                              "RIP_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_RIPaaFA",
                                                "fields": [
                                                    "ETA_NUM_EPMSI",
                                                    "RIP_NUM"
                                                ]
                                                }
                                          }
                                          ]

                                                  # Tables des ACE 
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='ACE':
      if table=="T_RIPaaCSTC":
        schema.descriptor['primaryKey']= [
            "ETA_NUM_EPMSI"
            ]
        schema.descriptor['foreignKeys']=[
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_BEN_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }
                                          
                                        ]

        

        
      #les autres tables : séjours RIP et ACE non individualisés, sejours et ACE à la fois

    elif tablestruct['structure'][tablestruct['tables'].index(table)] in ['Séjour (non individualisé)', 'ACE (non individualisé)','Séjour et ACE']:
      schema.descriptor['foreignKeys']= [{"fields":["ETA_NUM"],
                                              "reference": {
                                                "resource": "T_RIPaaE",
                                                "fields":["ETA_NUM"]
                                                }
                                        
                                              }
                                            
                                            ]


    # Les Résumés par actes ambulatoires  anonymisés

    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Externe':
      if table=="T_RIPaaR3A":
        schema.descriptor['primaryKey']= [
        "ETA_NUM_EPMSI",
        "ORD_NUM",
        "SEJ_IDT"
        ]
        schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM_EPMSI"],
                                           "reference": {
                                             "resource": "T_RIPaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
      else :
        schema.descriptor['foreignKeys']=[{"fields":[
                                                      "ETA_NUM_EPMSI",
                                                      "ORD_NUM",
                                                      "SEJ_IDT"
                                                                                        ],
                                          "reference": {
                                                "resource": "T_RIPaaR3A",
                                                "fields": [
                                                      "ETA_NUM_EPMSI",
                                                      "ORD_NUM",
                                                      "SEJ_IDT"
                                                ]
                                                }
                                         }
                                      ]

    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=tableexist['champ'][tableexist['table'].index(table)]
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files


<>:73: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:79: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:73: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:79: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:81: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-12-88ad74fe85f4>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'][i]=strdate(df['variables'][i])
<ipython-input-12-88ad74fe85f4>:73: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateDeleted'][z] is not '':
<ipython-input-12-88ad74fe85f4>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [5]:
from google.colab import files
!zip -r /content/sample_data.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.01.04/ (stored 0%)
  adding: content/.config/logs/2023.01.04/14.35.33.516796.log (deflated 55%)
  adding: content/.config/logs/2023.01.04/14.35.04.532501.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.03.867433.log (deflated 92%)
  adding: content/.config/logs/2023.01.04/14.35.34.388081.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.56.463059.log (deflated 86%)
  adding: content/.config/logs/2023.01.04/14.34.28.750487.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/conf

In [6]:
files.download('/content/sample_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>